In [117]:
import geopandas as gpd
import pandas as pd
import numpy as np
import random
import copy
import simpy
import networkx as nx
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

SECTION 1: PREPROCESSING & COST CALCULATIONS

(a) Load the raw OSM network data (exported in osm2pgrouting format)
    from a GeoPackage – here the layer is assumed to be named "edges".

(b) Load grid statistics (100m square data) from another GeoPackage.
    These grid cells include fields like "population", "offices", "businesses".

(c) We spatially join the grid stats to each edge (using its centroid)
    to compute a local congestion index. Then, based on the OSM “highway”
    type we assign a free‑flow speed and a priority factor.

In [118]:
# Load the network edges (adjust file name and layer as needed).
edges_gdf = gpd.read_file("groningen-stad-edges.gpkg", layer="edges")

# Compute the edge length (in meters) if not already available.
if "length" not in edges_gdf.columns:
    edges_gdf["length"] = edges_gdf.geometry.length

# Load the grid statistics (adjust file name, layer, and field names as needed).
grid_gdf = gpd.read_file("grid_stats.gpkg", layer="grid_stats")
# Add id column to the grid_gdf
grid_gdf["id"] = grid_gdf.index
# Compute the centroid of each edge.
edges_gdf["centroid"] = edges_gdf.geometry.centroid
centroids = gpd.GeoDataFrame(edges_gdf[["id"]], geometry=edges_gdf["centroid"], crs=edges_gdf.crs)

In [119]:
edges_gdf.head()
edges_gdf.set_index("id", inplace=False)

,osm_way_id,source,target,highway,base_speed,priority,oneway,length_m,cost,geometry,length,centroid
id,,,,,,,,,,,,
2,4464031,3,4,unclassified,30.0,3.0,0,278.877217,100.395798,"LINESTRING (232097.781 580026.024, 232112.632 ...",278.877217,POINT (232205.651 580113.137)
3,4694218,5,6,unclassified,60.0,3.0,0,297.006020,53.461084,"LINESTRING (229494.721 586385.619, 229526.469 ...",297.006020,POINT (229636.539 586345.724)
4,4694219,6,7,unclassified,60.0,3.0,0,40.220797,7.239743,"LINESTRING (229781.539 586332.55, 229821.582 5...",40.220797,POINT (229801.56 586334.442)
5,4748122,8,9,residential,30.0,1.8,0,50.884553,10.991064,"LINESTRING (233657.388 578848.762, 233609.467 ...",50.884553,POINT (233633.427 578840.206)
6,4773008,10,11,service,30.0,10.0,0,7.799109,9.358930,"LINESTRING (231810.57 586799.947, 231817.972 5...",7.799109,POINT (231814.271 586801.175)
...,...,...,...,...,...,...,...,...,...,...,...,...
96606,1355953709,68293,83547,service,30.0,10.0,0,66.899381,80.279258,"LINESTRING (239088.637 574374.686, 239082.427 ...",66.899381,POINT (239056.735 574368.268)
96607,1355953710,68314,83547,service,30.0,10.0,0,9.636410,11.563692,"LINESTRING (239025.781 574376.629, 239023.434 ...",9.636410,POINT (239024.607 574371.956)
96608,1355953710,83547,68287,service,30.0,10.0,0,56.340124,67.608149,"LINESTRING (239023.434 574367.282, 239017.583 ...",56.340124,POINT (238997.293 574357.816)


In [120]:
def check_network_connectivity(G):
    if nx.is_strongly_connected(G):
        print("The network is strongly connected.")
    else:
        print("The network is not strongly connected.")
        components = list(nx.strongly_connected_components(G))
        print(f"Number of strongly connected components: {len(components)}")
        print(f"Sizes of components: {[len(c) for c in components]}")

In [121]:
# Spatial join: for each edge (via its centroid) join the grid stats.
# (Using the predicate "within" – newer versions of GeoPandas support "predicate".)
joined = gpd.sjoin(centroids, grid_gdf, how="left", predicate="within")

# Check if 'id' column exists in joined DataFrame
if 'id' not in joined.columns:
    print("Warning: 'id' column not found in joined DataFrame. Using index as id.")
    joined['id'] = joined.index

# Select only the columns we need
joined_subset = joined[['id', 'population', 'offices', 'shops']]

# Merge with edges_gdf
edges_gdf = edges_gdf.merge(joined_subset, on="id", how="left")

for field in ["population", "offices", "shops"]:
    edges_gdf[field] = edges_gdf[field].fillna(0)
    # Make the ceiling 10000 for the original data.
    edges_gdf[field] = edges_gdf[field].clip(0, 10000)

# Define a function to compute a congestion index based on local grid stats.
def compute_congestion_index(population, offices, businesses):
    # We use arbitrary weights – adjust as needed.
    pop_factor = population / 1000.0
    office_factor = offices / 500.0
    business_factor = businesses / 500.0
    return 0.5 * pop_factor + 0.3 * office_factor + 0.2 * business_factor

edges_gdf["congestion_index"] = edges_gdf.apply(
    lambda row: compute_congestion_index(row["population"], row["offices"], row["shops"]),
    axis=1
)

In [122]:
# (Optional) You can smooth the congestion indices over adjacent edges.
def smooth_congestion_indices(edges_gdf, alpha=0.7, iterations=3):
    updated = edges_gdf.set_index("id")["congestion_index"].to_dict()
    # Build a node-to-edge mapping.
    node_to_edges = {}
    for _, row in edges_gdf.iterrows():
        src = row["source"]
        tgt = row["target"]
        node_to_edges.setdefault(src, []).append(row["id"])
        node_to_edges.setdefault(tgt, []).append(row["id"])
    for _ in range(iterations):
        new_vals = {}
        for _, row in edges_gdf.iterrows():
            eid = row["id"]
            src = row["source"]
            tgt = row["target"]
            neighbor_ids = set(node_to_edges.get(src, []) + node_to_edges.get(tgt, [])) - {eid}
            # For one-way edges, you might restrict neighbors (not done here).
            if neighbor_ids:
                neighbor_avg = np.mean([updated[nid] for nid in neighbor_ids])
                new_vals[eid] = alpha * updated[eid] + (1 - alpha) * neighbor_avg
            else:
                new_vals[eid] = updated[eid]
        updated = new_vals
    edges_gdf["smoothed_congestion_index"] = edges_gdf["id"].map(updated)
    return edges_gdf

edges_gdf = smooth_congestion_indices(edges_gdf, alpha=0.7, iterations=3)

SECTION 2: BUILDING THE GRAPH & INITIAL COST TABLE (per time-of-day)

Although our dynamic simulation will re‑compute travel times on the fly,
we first build a directed NetworkX graph using the “modified” edge data.

We also compute a “static” weight based on free‑flow travel time.
Additionally, we compute a capacity for each edge based on its length
and a per‑km density that depends on the road type.

In [123]:
# Compute capacity for each edge (vehicles per km * length in km).
road_capacity_density = {
    "motorway":      150,
    "motorway_link": 150,
    "trunk":         120,
    "trunk_link":    120,
    "primary":       100,
    "primary_link":  100,
    "secondary":     80,
    "secondary_link": 80,
    "tertiary":      60,
    "tertiary_link": 60,
    "residential":   40,
    "living_street": 40,
    "service":       30,
    "unclassified":  50,
}
default_density = 50

def compute_capacity(row):
    density = road_capacity_density.get(row["highway"], default_density)
    return (row["length"] / 1000.0) * density

edges_gdf["capacity"] = edges_gdf.apply(compute_capacity, axis=1)

# Calculate travel time in seconds
edges_gdf["travel_time_sec"] = edges_gdf["length"] / (edges_gdf["base_speed"] / 3.6)  # Convert km/h to m/s

# Build the directed graph G.
G = nx.DiGraph()
for _, row in edges_gdf.iterrows():
    # Add the forward edge
    G.add_edge(
        row["source"],
        row["target"],
        weight=row["travel_time_sec"],       # free‑flow travel time (static cost)
        edge_id=row["id"],
        travel_time_sec=row["travel_time_sec"],
        capacity=row["capacity"],
        length=row["length"],
        average_speed=row["base_speed"],
        oneway=row["oneway"]
    )
    
    # Add the reverse edge if it's not a one-way street (oneway == 0)
    if row["oneway"] == 0:
        G.add_edge(
            row["target"],
            row["source"],
            weight=row["travel_time_sec"],       # free‑flow travel time (static cost)
            edge_id=f"{row['id']}_rev",  # Add a suffix to distinguish reverse edges
            travel_time_sec=row["travel_time_sec"],
            capacity=row["capacity"],
            length=row["length"],
            average_speed=row["base_speed"],
            oneway=0
        )

# Build a list of nodes (used later for selecting origins/destinations).
nodes_list = list(G.nodes())

# For visualization later, create a mapping from edge_id to geometry.
edge_geometries = {row["id"]: row.geometry for _, row in edges_gdf.iterrows()}

SECTION 3: SIMULATION PARAMETERS & TIME-OF-DAY PATTERNS

We define a simulation start time (in seconds since midnight) and a total simulation
duration. Vehicles will be spawned at rates that depend on the time-of-day,
and realistic origin/destination patterns (e.g. morning: residential → office).

In [124]:
SIM_START = 6 * 3600        # e.g., simulation “clock” starts at 6:00 AM
SIM_DURATION = 6 * 3600     # simulate 6 hours (adjust as needed)

# Base parameters for vehicle generation.
BASE_BATCH_SIZE = 10         # base vehicles per batch
BASE_BATCH_INTERVAL = 60     # seconds between batches (5 minutes)

# Tagging nodes based on grid cell statistics
grid_sindex = grid_gdf.sindex

# Function to get the grid cell for a given point
def get_grid_cell(point):
    possible_matches_index = list(grid_sindex.intersection(point.bounds))
    possible_matches = grid_gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.contains(point)]
    if len(precise_matches) > 0:
        return precise_matches.iloc[0]
    return None

# Create a dictionary of node coordinates
node_coords = {}
for _, row in edges_gdf.iterrows():
    geom = row.geometry
    node_coords[row['source']] = Point(geom.coords[0])
    node_coords[row['target']] = Point(geom.coords[-1])

# Create a GeoDataFrame of nodes
nodes_gdf = gpd.GeoDataFrame(
    {'id': list(G.nodes())},
    geometry=[node_coords[node] for node in G.nodes()],
    crs=grid_gdf.crs
)

# Tag nodes based on grid cell statistics
residential_nodes = []
office_nodes = []

for _, node in nodes_gdf.iterrows():
    cell = get_grid_cell(node.geometry)
    if cell is not None:
        if cell['population'] > cell['offices'] * 2:  # More residential
            residential_nodes.append(node['id'])
        elif cell['offices'] > cell['population'] * 0.5:  # More office-oriented
            office_nodes.append(node['id'])
        else:  # Mixed use, randomly assign
            if random.random() < 0.5:
                residential_nodes.append(node['id'])
            else:
                office_nodes.append(node['id'])
    else:  # If node is not in any cell, randomly assign
        if random.random() < 0.7:  # Keeping the 70% residential ratio
            residential_nodes.append(node['id'])
        else:
            office_nodes.append(node['id'])

print(f"Tagged {len(residential_nodes)} residential nodes and {len(office_nodes)} office nodes.")

Tagged 7932 residential nodes and 8883 office nodes.


In [125]:
def get_time_period(sim_time):
    """Return a string (morning/day/evening/night) based on effective time-of-day."""
    effective = (SIM_START + sim_time) % 86400
    hour = effective // 3600
    if 6 <= hour < 10:
        return "morning"
    elif 16 <= hour < 20:
        return "evening"
    elif 10 <= hour < 16:
        return "day"
    else:
        return "night"

def get_spawn_multiplier(sim_time):
    """Return a multiplier for vehicle batch size based on time-of-day."""
    period = get_time_period(sim_time)
    if period in ["morning", "evening"]:
        return random.randint(2, 5)
    elif period == "night":
        return 0.5
    else:
        return random.randint(1, 2)

def get_origin_destination(sim_time):
    """
    Choose realistic origin and destination nodes based on time-of-day.
      - Morning: residential → office.
      - Evening: office → residential.
      - Otherwise: random.
    """
    period = get_time_period(sim_time)
    if period == "morning":
        return random.choice(residential_nodes), random.choice(office_nodes)
    elif period == "evening":
        return random.choice(office_nodes), random.choice(residential_nodes)
    else:
        return random.choice(nodes_list), random.choice(nodes_list)

def route_length(route):
    """Return total route length in meters (given a list of edge dictionaries)."""
    return sum(edge["length"] for edge in route)

SECTION 4: DYNAMIC CONGESTION FEEDBACK & RE‑ROUTING FUNCTIONS

A global dictionary (dynamic_occupancy) will keep track of the number
of vehicles on each edge at any time. We also record snapshots for visualization.

In [161]:
dynamic_occupancy = {edge_data["edge_id"]: 0 for u, v, edge_data in G.edges(data=True)}
occupancy_snapshots = []  # list of tuples: (simulation time, copy of occupancy dict)

# Parameters for the dynamic (BPR‑like) function.
alpha_dyn = 0.25
beta_dyn = 5.0

RE_ROUTE_THRESHOLD = 0.8   # if occupancy/capacity exceeds this, vehicles re-route

def occupancy_monitor(env, snapshot_interval=10):
    """Periodically record occupancy snapshots for visualization."""
    while True:
        occupancy_snapshots.append((env.now, copy.deepcopy(dynamic_occupancy)))
        yield env.timeout(snapshot_interval)

# -----------------------------------------------------------------------------
# Routing functions:
# (a) compute_route_for_vehicle: static (initial) route using Dijkstra.
# (b) compute_dynamic_route: re‑compute route on the fly using current occupancy.
# (c) compute_dynamic_route_snapshot: for route queries at a given time.
# -----------------------------------------------------------------------------

def compute_route_for_vehicle(G, source_node, destination):
    try:
        route_nodes = nx.shortest_path(G, source=source_node, target=destination, weight="weight")
    except nx.NetworkXNoPath:
        return None
    route_edges = []
    for u, v in zip(route_nodes[:-1], route_nodes[1:]):
        data = G.get_edge_data(u, v)
        route_edges.append({
            "edge_id": data["edge_id"],
            "from_node": u,
            "to_node": v,
            "travel_time_sec": data["travel_time_sec"],
            "capacity": data["capacity"],
            "length": data["length"],
            "average_speed": data["average_speed"]
        })
    return route_edges

def compute_dynamic_route(G, current_node, destination):
    """
    Build a temporary graph with updated weights using current dynamic occupancy.
    Weight = free_flow_time * (1 + alpha_dyn * ((occ + 1) / capacity) ** beta_dyn)
    """
    G_dynamic = nx.DiGraph()
    for u, v, data in G.edges(data=True):
        occ = dynamic_occupancy.get(data["edge_id"], 0)
        cap = data["capacity"]
        free_flow = data["travel_time_sec"]
        dynamic_weight = free_flow * (1 + alpha_dyn * ((occ + 1) / cap) ** beta_dyn)
        new_data = data.copy()
        new_data["weight"] = dynamic_weight
        G_dynamic.add_edge(u, v, **new_data)
    try:
        route_nodes = nx.shortest_path(G_dynamic, source=current_node, target=destination, weight="weight")
    except nx.NetworkXNoPath:
        return None
    route_edges = []
    for u, v in zip(route_nodes[:-1], route_nodes[1:]):
        data = G.get_edge_data(u, v)
        route_edges.append({
            "edge_id": data["edge_id"],
            "from_node": u,
            "to_node": v,
            "travel_time_sec": data["travel_time_sec"],
            "capacity": data["capacity"],
            "length": data["length"],
            "average_speed": data["average_speed"]
        })
    return route_edges

def compute_dynamic_route_snapshot(G, current_node, destination, occ_snapshot):
    """
    Like compute_dynamic_route() but uses a provided occupancy snapshot.
    """
    G_dynamic = nx.DiGraph()
    for u, v, data in G.edges(data=True):
        occ = occ_snapshot.get(data["edge_id"], 0)
        cap = data["capacity"]
        free_flow = data["travel_time_sec"]
        dynamic_weight = free_flow * (1 + alpha_dyn * ((occ + 1) / cap) ** beta_dyn)
        new_data = data.copy()
        new_data["weight"] = dynamic_weight
        G_dynamic.add_edge(u, v, **new_data)
    try:
        route_nodes = nx.shortest_path(G_dynamic, source=current_node, target=destination, weight="weight")
    except nx.NetworkXNoPath:
        return None
    route_edges = []
    for u, v in zip(route_nodes[:-1], route_nodes[1:]):
        data = G.get_edge_data(u, v)
        route_edges.append({
            "edge_id": data["edge_id"],
            "from_node": u,
            "to_node": v,
            "travel_time_sec": data["travel_time_sec"],
            "capacity": data["capacity"],
            "length": data["length"],
            "average_speed": data["average_speed"]
        })
    return route_edges
    # routes = list(nx.shortest_simple_paths(G_dynamic, current_node, destination, weight="weight"))
    # best_route = None
    # best_time = float('inf')
    # k = 5
    
    # for route_nodes in routes[:k]:  # Consider top k routes
    #     route_edges = []
    #     total_time = 0
    #     for u, v in zip(route_nodes[:-1], route_nodes[1:]):
    #         data = G.get_edge_data(u, v)
    #         edge_time = data["travel_time_sec"] * (1 + alpha_dyn * ((occ_snapshot.get(data["edge_id"], 0) + 1) / data["capacity"]) ** beta_dyn)
    #         total_time += edge_time
    #         route_edges.append({
    #             "edge_id": data["edge_id"],
    #             "from_node": u,
    #             "to_node": v,
    #             "travel_time_sec": data["travel_time_sec"],
    #             "capacity": data["capacity"],
    #             "length": data["length"],
    #             "average_speed": data["average_speed"]
    #         })
    #     if total_time < best_time:
    #         best_time = total_time
    #         best_route = route_edges
    
    # return best_route

SECTION 5: VEHICLE PROCESS & GENERATION

Each vehicle follows its route edge-by-edge. Before entering an edge,
it checks if the occupancy fraction exceeds the threshold. If so, it re-computes
a new route from its current node to the destination.
In addition, we enforce that the route must be at least 2 km long.

In [127]:
def vehicle_process(env, vehicle_id, route_edges, destination):
    """
    Each vehicle traverses its route edge-by-edge.
    Before entering an edge, if its occupancy fraction exceeds the threshold,
    the vehicle re-computes a new route from its current node to the destination.
    To avoid infinite re-routing loops, we limit the number of consecutive re-routes.
    """
    if not route_edges:
        return
    current_node = route_edges[0]["from_node"]
    i = 0
    reroute_attempts = 0
    MAX_REROUTES = 3  # maximum number of re-route attempts before proceeding
    while i < len(route_edges):
        edge = route_edges[i]
        occ = dynamic_occupancy.get(edge["edge_id"], 0)
        cap = edge["capacity"]
        # If the occupancy is above the threshold and we haven't exceeded max re-routes...
        if (occ / cap) > RE_ROUTE_THRESHOLD:
            if reroute_attempts < MAX_REROUTES:
                new_route = compute_dynamic_route(G, current_node, destination)
                if new_route is not None:
                    print(f"[Time {env.now:5.1f} sec] Vehicle {vehicle_id} re-routing from node {current_node} due to congestion on edge {edge['edge_id']}.")
                    route_edges = new_route  # adopt the new route
                    i = 0  # reset to the start of the new route
                    reroute_attempts += 1
                    continue  # re-check the first edge of the new route
            else:
                # Maximum re-route attempts reached; proceed on current route despite congestion.
                print(f"[Time {env.now:5.1f} sec] Vehicle {vehicle_id} reached maximum re-route attempts at node {current_node}. Proceeding on current route.")
        # Reset the counter once an edge is successfully entered.
        reroute_attempts = 0
        # Vehicle enters the edge.
        dynamic_occupancy[edge["edge_id"]] += 1
        occ_new = dynamic_occupancy[edge["edge_id"]]
        free_flow_time = edge["travel_time_sec"]
        travel_time = free_flow_time * (1 + alpha_dyn * ((occ_new) / cap) ** beta_dyn)
        yield env.timeout(travel_time)
        dynamic_occupancy[edge["edge_id"]] -= 1
        current_node = edge["to_node"]
        i += 1
    print(f"[Time {env.now:5.1f} sec] Vehicle {vehicle_id} reached destination {destination}.")

def vehicle_generator(env):
    """
    Generate vehicles in batches. Batch size is modulated by time-of-day.
    Each vehicle’s origin and destination are chosen based on realistic patterns.
    Only routes with a total length of at least 2 km are accepted.
    """
    vehicle_id = 0
    while env.now < SIM_DURATION:
        multiplier = get_spawn_multiplier(env.now)
        batch_size = int(BASE_BATCH_SIZE * multiplier)
        for _ in range(batch_size):
            source, destination = get_origin_destination(env.now)
            if source == destination:
                continue
            attempts = 0
            route = None
            while attempts < 10:
                route = compute_route_for_vehicle(G, source, destination)
                if route and (route_length(route) >= 2000):
                    break
                source, destination = get_origin_destination(env.now)
                attempts += 1
            if route is None or route_length(route) < 2000:
                continue
            env.process(vehicle_process(env, vehicle_id, route, destination))
            vehicle_id += 1
        print(f"[Sim Time {env.now:5.1f} sec] Launched batch of {batch_size} vehicles.")
        yield env.timeout(BASE_BATCH_INTERVAL)

def run_simulation():
    """
    Set up and run the SimPy environment:
      - Start occupancy monitoring.
      - Start vehicle generation.
      - Run until SIM_DURATION.
    """
    env = simpy.Environment()
    env.process(occupancy_monitor(env, snapshot_interval=10))
    env.process(vehicle_generator(env))
    env.run(until=SIM_DURATION)
    print("Simulation complete.")

SECTION 6: ROUTE QUERY FUNCTIONALITY

These functions allow you to “freeze” the dynamic state at a given time
(using recorded occupancy snapshots) and compute a route for a given OD pair.

In [128]:
def query_route_at_time(query_time, source, destination):
    if not occupancy_snapshots:
        print("No occupancy snapshots available.")
        return None
    closest_snapshot = min(occupancy_snapshots, key=lambda snap: abs(snap[0] - query_time))
    snap_time, occ_snapshot = closest_snapshot
    print(f"Query time: {query_time} sec; using occupancy snapshot at {snap_time} sec.")
    route = compute_dynamic_route_snapshot(G, source, destination, occ_snapshot)
    
    if route:
        # Calculate the estimated travel time
        total_time = sum(
            edge["travel_time_sec"] * (1 + alpha_dyn * ((occ_snapshot.get(edge["edge_id"], 0) + 1) / edge["capacity"]) ** beta_dyn)
            for edge in route
        )
        return route, total_time
    return None, None

def query_routes_at_intervals(source, destination, interval=3600, total_duration=SIM_DURATION):
    queries = []
    t = 0
    while t <= total_duration:
        route, travel_time = query_route_at_time(t, source, destination)
        queries.append((t, route, travel_time))
        t += interval
    return queries

In [129]:
# Run the dynamic simulation.
run_simulation()

[Sim Time   0.0 sec] Launched batch of 20 vehicles.
[Sim Time  60.0 sec] Launched batch of 20 vehicles.
[Time  72.5 sec] Vehicle 15 re-routing from node 32154 due to congestion on edge 31995_rev.
[Sim Time 120.0 sec] Launched batch of 40 vehicles.
[Time 144.6 sec] Vehicle 55 re-routing from node 32308 due to congestion on edge 46474.
[Time 161.2 sec] Vehicle 40 re-routing from node 68319 due to congestion on edge 84583_rev.
[Sim Time 180.0 sec] Launched batch of 50 vehicles.
[Time 182.4 sec] Vehicle 67 re-routing from node 74768 due to congestion on edge 80337.
[Time 208.0 sec] Vehicle 29 reached destination 74304.
[Time 214.7 sec] Vehicle 101 re-routing from node 48668 due to congestion on edge 80337_rev.
[Time 216.4 sec] Vehicle 36 re-routing from node 48663 due to congestion on edge 49073_rev.
[Time 232.9 sec] Vehicle 1 reached destination 32402.
[Sim Time 240.0 sec] Launched batch of 50 vehicles.
[Time 261.0 sec] Vehicle 171 re-routing from node 7198 due to congestion on edge 6878.

In [ ]:
# Save occupancy snapshots to a df and save it to a csv file
occupancy_snapshots_df = pd.DataFrame(occupancy_snapshots)
occupancy_snapshots_df.to_csv("occupancy_snapshots.csv", index=False)

In [165]:
if residential_nodes and office_nodes:
    query_source = random.choice(residential_nodes)
    query_destination = random.choice(office_nodes)
    print(f"\nQuerying routes from residential node {query_source} to office node {query_destination} at hourly intervals:")
    route_queries = query_routes_at_intervals(query_source, query_destination, interval=600, total_duration=SIM_DURATION)
    for query_time, route, travel_time in route_queries:
        if route is not None:
            total_length = route_length(route) / 1000.0
            print(f"  At {query_time/3600:.1f} hr: Route length = {total_length:.2f} km; Estimated travel time = {travel_time/60:.2f} min; edges: {[edge['edge_id'] for edge in route]}")
        else:
            print(f"  At {query_time/3600:.1f} hr: No route found.")
else:
    print("Not enough nodes for realistic route queries.")


Querying routes from residential node 31130 to office node 33783 at hourly intervals:
No occupancy snapshots available.


TypeError: cannot unpack non-iterable NoneType object

 SECTION 7: VISUALIZATION (ANIMATION OF OCCUPANCY)
 
 We animate the network by coloring each edge according to its occupancy fraction.

In [139]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
from PIL import Image
import io

def animate_network(edges_gdf, occupancy_snapshots, interval=200):

    # Build the network segments, edge IDs, and capacities
    segments = []
    edge_ids = []
    capacities = []
    for _, row in edges_gdf.iterrows():
        if row.geometry.geom_type == 'LineString':
            segments.append(list(row.geometry.coords))
            edge_ids.append(row["id"])
            capacities.append(row["capacity"])
        elif row.geometry.geom_type == 'MultiLineString':
            for line in row.geometry:
                segments.append(list(line.coords))
                edge_ids.append(row["id"])
                capacities.append(row["capacity"])
        # Skip other geometry types

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_aspect('equal')
    ax.set_title("Dynamic Network Congestion")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    # Add background network with thin gray lines
    background_lc = LineCollection(segments, colors='lightgray', linewidths=1.2, zorder=1)
    ax.add_collection(background_lc)

    norm = plt.Normalize(0, 1)
    lc = LineCollection(segments, cmap='Reds', norm=norm, zorder=2)
    lc.set_array(np.zeros(len(segments)))
    lc.set_linewidth(1)
    ax.add_collection(lc)
    
    all_coords = np.concatenate(segments)
    ax.set_xlim(np.min(all_coords[:, 0]) - 0.01, np.max(all_coords[:, 0]) + 0.01)
    ax.set_ylim(np.min(all_coords[:, 1]) - 0.01, np.max(all_coords[:, 1]) + 0.01)
    cbar = plt.colorbar(lc, ax=ax)
    cbar.set_label("Occupancy Fraction (occupancy / capacity)")
    time_text = ax.text(0.02, 0.95, "", transform=ax.transAxes, fontsize=12, color="blue")
    capacity_dict = {eid: cap for eid, cap in zip(edge_ids, capacities)}

    # --- Filter snapshots based on the desired time interval ---
    filtered_snapshots = []
    last_frame_time = -np.inf
    for sim_time, occ_dict in occupancy_snapshots:
        if sim_time - last_frame_time >= interval:
            filtered_snapshots.append((sim_time, occ_dict))
            last_frame_time = sim_time

    frames = []
    for sim_time, occ_dict in filtered_snapshots:
        # Compute occupancy fractions for each edge
        occ_fractions = []
        for eid in edge_ids:
            occ = occ_dict.get(eid, 0)
            cap = capacity_dict.get(eid, 1)
            fraction = min(occ / cap, 1.0)
            occ_fractions.append(fraction)
        lc.set_array(np.array(occ_fractions))
        time_text.set_text(f"Sim Time: {sim_time:.1f} sec")
        
        # Render the figure to a buffer and create an image
        fig.canvas.draw()
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=300, bbox_inches='tight', pad_inches=0.1)
        buf.seek(0)
        img = Image.open(buf)
        frames.append(img.copy())
        buf.close()
    
    plt.close(fig)
    return frames

# Create the animation frames using the filtered snapshots
frames = animate_network(edges_gdf, occupancy_snapshots, interval=200)

# Save the frames as a GIF with the specified display duration for each frame
frames[0].save("congestion.gif", save_all=True, append_images=frames[1:], duration=200, loop=0)

print("Animation saved as congestion.gif")

Animation saved as congestion.gif
